In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
df = pd.read_csv('16P.csv',engine='python', encoding='cp1252')
df.drop("Response Id",axis=1, inplace=True)
df.replace(["ESTJ","ENTJ","ESFJ","ENFJ","ISTJ","ISFJ","INTJ","INFJ","ESTP","ESFP","ENTP","ENFP","ISTP","ISFP","INTP","INFP"],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],inplace=True)
#Normalization
df_np= df.to_numpy()
df_np_n = df.to_numpy()
new=df_np_n[:,:-1]+3
df_np_n=new/6

In [ ]:
def most_frequent(List):
    counter = 0
    num = List[0]
     
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
 
    return num

In [ ]:
def knn_algorithm(k,N):
    print("All wrong predictions and their indexes and their acculy value")
    if N:
        df_np = df_np_n
    else:
        df = pd.read_csv('16P.csv',engine='python', encoding='cp1252')
        df.drop("Response Id",axis=1, inplace=True)
        df.replace(["ESTJ","ENTJ","ESFJ","ENFJ","ISTJ","ISFJ","INTJ","INFJ","ESTP","ESFP","ENTP","ENFP","ISTP","ISFP","INTP","INFP"],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],inplace=True)
        df_np=df.to_numpy()
    train_set, test_set = np.split(df_np, [48001,])
#k calculate distances
    x2 = np.sum(test_set**2,axis=1) 
    y2 = np.sum(train_set**2,axis=1)
    xy = np.dot(test_set, train_set.T)
    x2 = x2.reshape(-1, 1)
    dists = np.sqrt(x2 - 2*xy + y2)
    argsort = np.argsort(dists) # I found the indexes of its neighbors for each data in the test set in the form of a 2-dimensional numpy array.
    
# make prediction
    indexler=argsort[:,:k]  # i'm taking k neighbors
    #  i'm installing the main array here from scratch because the code was corrupted before
    df = pd.read_csv('16P.csv',engine='python', encoding='cp1252')
    df.drop("Response Id",axis=1, inplace=True)
    df.replace(["ESTJ","ENTJ","ESFJ","ENFJ","ISTJ","ISFJ","INTJ","INFJ","ESTP","ESFP","ENTP","ENFP","ISTP","ISFP","INTP","INFP"],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],inplace=True)
    df = df.to_numpy()
    train,test=np.split(df, [48001,])
    matrix_arrayi=np.zeros((16,16))
    for y,x in enumerate(indexler,0):
        tahmin_listesi=[]
        for index in x:
            tahmin_listesi.append(df[index,-1])
        prediction = most_frequent(tahmin_listesi)
        if prediction == test[y,-1] :
            matrix_arrayi[prediction][prediction]= matrix_arrayi[prediction][prediction]+1
        else:
            print((y+48001),"Prediction:",prediction,"Actual:",test[y,-1])
            matrix_arrayi[test[y,-1]][prediction]=matrix_arrayi[test[y,-1]][prediction]+1
    accuracy = 0
    precision = 0
    recall = 0
# find TP FP FN FP values for multiple classes
    for x in range(16):
        TP = matrix_arrayi[x][x]
        TN = matrix_arrayi.sum() - np.sum(matrix_arrayi,axis=1)[x] - np.sum(matrix_arrayi,axis=0)[x]
        FP = np.sum(matrix_arrayi,axis=0)[x] - matrix_arrayi[x][x]
        FN = np.sum(matrix_arrayi,axis=1)[x] - matrix_arrayi[x][x]
        xclas_accuracy= (TP + TN )/ matrix_arrayi.sum()
        xclas_precision = TP / (TP + FP)
        xclas_recall = TP / (TP + FN )
        accuracy =accuracy+ xclas_accuracy
        precision =precision + xclas_precision
        recall =recall + xclas_recall
        
    accuracy= accuracy / 16
    precision = precision / 16
    recall = recall / 16
    print("Accuracy: ",accuracy)
    print ("Precision: ",precision)
    print("Recall: ",recall)
    

# If the value of N is T (which means True), normalization is performed. If F is (False), normalization is not performed.    
knn_algorithm(3,False)

Note: I divided the 60k data set into two parts of 48000 and 12000. I did not use cross validation

Error Analysis for Classification
--------------------------------------------------

FEW MISCLASSIFIED SAMPLES: 

1- index(52296) prediction(8) actual(2) when k = 3   
    My program evaluated the data with an index of 52296 as 8 (ESTP), but the correct personality was 2 (ESFJ).
    The 3 closest neighbors of this person are: [8, 8, 8] so [ESTP, ESTP,ESTP]
    My Opinion For This Situation: All 3 of the nearest neighbors are ESTP, perhaps the person in this index may be acting as a
    boundary between the two classes (i.e., at the end of classes ESFJ and ESTP).
    
2- index(55535), prediction(4), actual(14)
    My program evaluated the data with an index of 55535 as 4 (ISTJ), but the correct personality was 14 (INTP).
    The 5 closest neighbors of this person are: [4, 4, 14, 14, 12] so [ISTJ,ISTJ,INTP,INTP,ISTP]
    My Opinion For This Situation :
    The reason for the incorrect prediction of the personality of the person in this index: My algorithm is a field algorithm 
    that selects which is the closest personality as a prediction if there are two same number of personalities. so in fact, in 
    this case, the value of k is treated as if it is equal to 1.  This also causes an incorrect estimate.

3- index(54156), prediction(12), actual(2) 
     My program evaluated the data with an index of 55535 as 12 (ISTP), but the correct personality was 4 (ISTJ).
     The 3 closest neighbors of this person are: [12, 2, 4] so [ISTP,ESFJ,ISTJ]
     My Opinion For This Situation :
     If our k parameter were 5 instead of 3, we could get two from any value in the list. That way, we would be more likely to 
     make the right prediction. Because there is no case of any misclassification error in the outputs where the value of k is
     5, and all 5 values are different from each other in the prediction list.

4- index(50632), prediction(13), actual(3) 
    My program evaluated the data with an index of 50632 as 13 (ISFP), but the correct personality was 3 (ENFJ).
    The 9 closest neighbors of this person are: [13, 13, 13, 13, 13, 13, 13, 13, 13] so [ISFP,...,ISFP]
    My Opinion For This Situation :
    This may be due to the reason I gave in the first misclassification example, or it may be because I did not use the k-fold 
    algorithm.

------------------------------------------------

k=1
accuracy:  0.9361351891981997
precision: 0.9781342923350658
recall:    0.9781238169299069

k=3
accuracy:  0.9367602933822302
precision: 0.9881567079070425
recall:    0.9881318663282883

k=5
accuracy:  0.9368019669944991
precision: 0.9888218597978752
recall:    0.9888124342592615

k=7
accuracy:  0.9368175945990997
precision: 0.9890748310420907
recall:    0.989064728846451

k=9
accuracy 0.9368175945990997
precision 0.9890799935341432
recall 0.9890669396550056

k=21
accuracy:  0.9368332222037006
precision: 0.9893214904642028
recall:    0.9893158360146274

Performance Analysis For k :
    As the value of k increased, I observed increases in all my score values (accuracy,precision,recall) close to 0.001 or 
    0,0001. While the value of k is 3, the total misclassification error is 130-odd, and while k = 5, the total 
    misclassification error is 130-odd. I was expecting the number of errors to decrease as the k value increases, the reason 
    why it does not decrease may be because I have not written a k-fold algorithm in my Knn algorithm.

Performance Analysis In My KNN Algorithm: 
    The part that used the computer RAM the most was the code part of distance calculation. While calculating the distance, I
    found that the 16GB RAM is used up to 98%. But when I ran the same code on a computer with 8 RAM, I found that 98% of the 
    RAM was used.  I was surprised not to get it when I was expecting to get a memory error on an 8 RAM computer. I'm 
    stillinvestigating why.

Performance Analysis for K-Fold : 
    Although I couldn't use the K-fold algorithm in my code, I did a lot of research about . Therefore, if I used this 
    algorithm, I can predict that my score values would increase by 0.0001.

Accuracy, Precision and Recall : 
    As I shared above, scores are very ideal for a general Knn algorithm.
    
Effect of Normalization : 
    k=3   with Normalization
    accuracy:  0.9367602933822302
    precision: 0.9881567079070425
    recall:    0.9881318663282883
    
    k=3   without Normalization :
    accuracy:  0.9367498749791632
    precision: 0.9879904070574
    recall:    0.9879649625646864
    I have observed Decays around 0.005 between the data I normalized and did not.    
    I observed a decrease of around 0.005 between the data that I normalized and the data that I did not.

 Note: After implanting the normalization code into my knn algorithm function, I observed an increase in all my scores     
     k=3 with Normalization ( implamented in Knn_algorithm function )
     accuracy:  0.9369634522420405
     precision: 0.9913765948308326
     recall:    0.9914025958204421